# PetMind : Pet Supplies

PetMind is a retailer of products for pets. They are based in the United States.

PetMind sells products that are a mix of luxury items and everyday items. Luxury items include toys. Everyday items include food.

The company wants to increase sales by selling more products for some animals repeatedly.

They have been testing this approach for the last year.

They now want a report on how repeat purchases impact sales.

## Data

The data is available in the table `pet_supplies`.

The dataset contains the sales records in the stores last year.

| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
|product_id | Nominal. The unique identifier of the product. </br>Missing values are not possible due to the database structure.|
| category | Nominal. The category of the product, one of 6 values (Housing, Food, Toys, Equipment, Medicine, Accessory). </br>Missing values should be replaced with “Unknown”. |
| animal | Nominal. The type of animal the product is for. One of Dog, Cat, Fish, Bird. </br>Missing values should be replaced with “Unknown”. |
| size | Ordinal. The size of animal the product is for. Small, Medium, Large. </br>Missing values should be replaced with “Unknown”.|
| price | Continuous. The price the product is sold at. Can be any positive value, round to 2 decimal places. </br>Missing values should be replaced with the overall median price. |
| sales | Continuous. The value of all sales of the product in the last year. This can be any positive value, rounded to 2 decimal places. </br>Missing values should be replaced with the overall median sales. |
| rating | Discrete. Customer rating of the product from 1 to 10. </br>Missing values should be replaced with 0. |
| repeat_purchase | Nominal. Whether customers repeatedly buy the product (1) or not (0). </br>Missing values should be removed. |

# Task 1

From taking a quick look at the data, you are pretty certain it isn't quite as it should be. You need to make sure all of the data is clean before you start your analysis. The table below shows what the data should look like. 

Write a query to return a table that matches the description provided.

Do not update the original table. 

| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
|product_id | Nominal. The unique identifier of the product. </br>Missing values are not possible due to the database structure.|
| category | Nominal. The category of the product, one of 6 values (Housing, Food, Toys, Equipment, Medicine, Accessory). </br>Missing values should be replaced with “Unknown”. |
| animal | Nominal. The type of animal the product is for. One of Dog, Cat, Fish, Bird. </br>Missing values should be replaced with “Unknown”. |
| size | Ordinal. The size of animal the product is for. Small, Medium, Large. </br>Missing values should be replaced with “Unknown”.|
| price | Continuous. The price the product is sold at. Can be any positive value, round to 2 decimal places. </br>Missing values should be replaced with 0. |
| sales | Continuous. The value of all sales of the product in the last year. This can be any positive value, rounded to 2 decimal places. </br>Missing values should be replaced with the overall median sales. |
| rating | Discrete. Customer rating of the product from 1 to 10. </br>Missing values should be replaced with 0. |
| repeat_purchase | Nominal. Whether customers repeatedly buy the product (1) or not (0). </br>Missing values should be removed. |

In [18]:
WITH
  median_sales AS (
    SELECT
      PERCENTILE_CONT(0.5) WITHIN GROUP (
        ORDER BY
          sales
      ) AS median_sales
    FROM
      pet_supplies
    WHERE
      sales IS NOT NULL
  )
SELECT
  product_id,
  CASE
    WHEN LOWER(category) IN (
      'housing',
      'food',
      'toys',
      'equipment',
      'medicine',
      'accessory'
    ) THEN INITCAP(category)
    ELSE 'Unknown'
  END AS category,
  CASE
    WHEN LOWER(animal) IN ('dog', 'cat', 'fish', 'bird') THEN INITCAP(animal)
    ELSE 'Unknown'
  END AS animal,
  CASE
    WHEN LOWER(size) IN ('small', 'medium', 'large') THEN INITCAP(size)
    ELSE 'Unknown'
  END AS size,
  ROUND(
    COALESCE(
      CASE
        WHEN price ~ '^[0-9]+(\.[0-9]+)?$' THEN price::NUMERIC
        ELSE NULL
      END,
      0
    )::NUMERIC,
    2
  ) AS price,
  ROUND(
    COALESCE(
      sales,
      (
        SELECT
          median_sales
        FROM
          median_sales
      )
    )::NUMERIC,
    2
  ) AS sales,
  COALESCE(rating, 0) AS rating,
  repeat_purchase
FROM
  pet_supplies
WHERE
  repeat_purchase IS NOT NULL;

,product_id,category,animal,size,price,sales,rating,repeat_purchase
0,1,Food,Bird,Large,51.10,1860.62,7,1
1,2,Housing,Bird,Medium,35.98,963.60,6,0
2,3,Food,Dog,Medium,31.23,898.30,5,1
3,4,Medicine,Cat,Small,24.95,982.15,6,1
4,5,Housing,Cat,Small,26.18,832.63,7,1
...,...,...,...,...,...,...,...,...
1495,1496,Food,Bird,Medium,40.91,1203.36,5,0
1496,1497,Equipment,Dog,Small,32.89,1099.97,5,0
1497,1498,Food,Fish,Small,25.79,724.02,3,0
1498,1499,Medicine,Fish,Small,19.94,658.66,5,1


# Task 2

You want to show whether sales are higher for repeat purchases for different animals. You also want to give a range for the sales.

Write a query to return the `animal`, `repeat_purchase` indicator and the `avg_sales`, along with the `min_sales` and `max_sales`. All values should be rounded to whole numbers. 

You should use the original `pet_supplies` data for this task. 

In [19]:
SELECT
  animal,
  repeat_purchase,
  ROUND(AVG(sales)::NUMERIC) AS avg_sales,
  ROUND(MIN(sales)::NUMERIC) AS min_sales,
  ROUND(MAX(sales)::NUMERIC) AS max_sales
FROM
  pet_supplies
GROUP BY
  animal,
  repeat_purchase
ORDER BY
  animal,
  repeat_purchase;

,animal,repeat_purchase,avg_sales,min_sales,max_sales
0,Bird,0,1380,858,2255
1,Bird,1,1408,853,2256
2,Cat,0,1035,512,1730
3,Cat,1,998,512,1724
4,Dog,0,1084,574,1795
5,Dog,1,1038,574,1797
6,Fish,0,705,288,1307
7,Fish,1,693,287,1301


# Task 3

The management team want to focus on efforts in the next year on the most popular pets - cats and dogs - for products that are bought repeatedly. 

Write a query to return the `product_id`, `sales` and `rating` for the relevant products.

You should use the original `pet_supplies` data for this task.

In [20]:
SELECT
  product_id,
  sales,
  rating
FROM
  pet_supplies
WHERE
  LOWER(animal) IN ('cat', 'dog')
  AND repeat_purchase = 1;

,product_id,sales,rating
0,3,898.30,5.0
1,4,982.15,6.0
2,5,832.63,7.0
3,11,1457.22,7.0
4,14,1450.50,6.0
...,...,...,...
547,1485,1039.06,6.0
548,1486,615.22,3.0
549,1491,746.16,7.0
550,1493,993.08,7.0


MIT License

Copyright © [Vinodh Zamboulingame](https://github.com/vzamboulingame/data-portfolio/blob/main/LICENSE)